In [1]:
import importlib
import dbops
import cv2
import time
import numpy as np
from tqdm import tqdm
from PIL import Image

import torch
from torchvision.transforms import functional as F

from facelib import EmotionDetector

import glob
import math

/scratch/u1319464/miniforge3/envs/ds4150-faces/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
importlib.reload(dbops)  # Reload the module from source

<module 'dbops' from '/scratch/u1319464/Data-Mining-Spring-2025-Project/dbops.py'>

In [3]:
emotion_detector = EmotionDetector()

from EmotionDetector: weights loaded


In [28]:

EMB_METHOD = 'FaceLib.EmotionDetector(name="densnet121")'

# from facelib/FacialExpressions.py
labels = np.array(['angry', 'disgust', 'fear', 'happy', 'sad', 'surprise', 'neutral'])

def do_embedding(faces):
    if len(faces) > 0:
        with torch.no_grad():
            # [0,1], BGR <-- very important
            faces = faces[:,[2,1,0]].float().div(255).to(emotion_detector.device)
            # use this to override EmotionDetector.detect_emotion. It does the exact same thing, but returns the embedding in addition to classification
            emotions = emotion_detector.model(faces)
            prob = torch.softmax(emotions, dim=1)
            emo_prob, emo_idx = torch.max(prob, dim=1)
            return emotions, labels[emo_idx.tolist()], emo_prob.tolist()
    else:
        raise Exception('no faces to parse')


In [29]:
from IPython.display import display

In [30]:


for movie_ref in dbops.Movie.iterate_all_movies():
    imdb_id = movie_ref.get_imdb_id()
    title = movie_ref.get_title()
    file_path = movie_ref.get_file_path()

    for frame_ref in tqdm(movie_ref.iterate_frames(), desc=movie_ref.get_title()):
        
        for face_ref in frame_ref.iterate_faces():

            emb_ref = dbops.FaceEmotionEmbedding.add_face_emotion(face_ref.imdb_id, face_ref.frame_index, face_ref.face_index, EMB_METHOD)

            try:
            
                aligned_face_img = face_ref.get_face_image_aligned()

                embeddings, classifications, confidences = do_embedding(F.pil_to_tensor(aligned_face_img).unsqueeze(0))

                # print(classifications, confidences)
                # display(aligned_face_img)
                # display(Image.fromarray(cv2.imread(aligned_face_img.filename)))
                
                emb_ref.set_embedding(embeddings[0].cpu().numpy(), classifications[0], confidences[0])

                # print(emb_ref.get_embedding())

            except Exception as e:
                print(e)


The Invisible Man (1933): 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████| 2263/2263 [01:06<00:00, 34.15it/s]
